In [ ]:
from dataloc import *
from bgs_helpers import *
from astropy.table import Table

In [ ]:
# Creates SV3 FUJI merged file (no Y3 supplement yet)
create_merged_file(BGS_SV3_ANY_FULL_FILE, IAN_BGS_SV3_MERGED_NOY3_FILE, 'sv3', photoz_wspec=False)

# Now remove galaxies in the patches of SV3 that have poor overlap with Y3
sv3_table: Table = Table.read(IAN_BGS_SV3_MERGED_NOY3_FILE, format='fits')
print(len(sv3_table))

sv3_table['region'] = tile_to_region(sv3_table['NEAREST_TILEIDS'][:,0])
to_remove = np.isin(sv3_table['region'], sv3_poor_y3overlap)
sv3_table.remove_rows(to_remove)
print(len(sv3_table))

sv3_table.write(IAN_BGS_SV3_MERGED_NOY3_FILE, format='fits', overwrite='True')

In [ ]:
# Creates Y1 IRON merged file
create_merged_file(BGS_Y1_ANY_FULL_FILE, IAN_BGS_Y1_MERGED_FILE, "1")

In [ ]:
# Creates Y3 LOA merged file
create_merged_file(BGS_Y3_ANY_FULL_FILE, IAN_BGS_Y3_MERGED_FILE_LOA, "3")

# Add columns to allow the possibility to make a Y3 catalog that is cut to SV3 regions.
table = Table.read(IAN_BGS_Y3_MERGED_FILE_LOA, format='fits')
sv3tiles = read_tiles_Y3_sv3()
ntiles_inside, nearest_tile_ids = find_tiles_for_galaxies(sv3tiles, table_to_df(table), 10)
if 'NTILE_MINE_SV3' in table.columns:
    table.remove_columns(['NTILE_MINE_SV3'])
if 'NEAREST_TILEIDS_SV3' in table.columns:
    table.remove_columns(['NEAREST_TILEIDS_SV3'])
table.add_column(ntiles_inside, name="NTILE_MINE_SV3")
table.add_column(nearest_tile_ids, name="NEAREST_TILEIDS_SV3")

table.write(IAN_BGS_Y3_MERGED_FILE_LOA, format='fits', overwrite=True)

In [ ]:
# Now add Y3 galaxies to supplement the NN catalog, especialy valuable at the edges of the regions
# They won't go into the main catalog because their NTILE_MINE is < 10
supplement_sv3_merged_file_with_y3(IAN_BGS_SV3_MERGED_NOY3_FILE, IAN_BGS_Y3_MERGED_FILE_LOA, IAN_BGS_SV3_MERGED_FILE)

In [ ]:
# Make a variant of Y3 Loa cut to SV3 regions, for the Fiber Incompleteness Study

# First replace photo-z column with one that has no spec-z 'contamination' 
table = add_photz_columns(IAN_BGS_Y3_MERGED_FILE_LOA, IAN_PHOT_Z_FILE_NOSPEC)

# Remove galaxies that are far away from the SV3 regions
keep = table['NTILE_MINE_SV3'] >= 1 # Not going to 10+ here because we want to keep nearby galaxies for the NN catalog

print(f"Y3 Loa table cut from {len(table)} to {keep.sum()} rows for like-SV3 version")
table  = table[keep]

table.write(IAN_BGS_Y3_MERGED_FILE_LOA_SV3CUT, format='fits', overwrite=True)


## For updating just the Quiescent column of the merged files

In [ ]:

for file in [IAN_BGS_Y1_MERGED_FILE, IAN_BGS_Y3_MERGED_FILE_LOA, IAN_BGS_Y3_MERGED_FILE_LOA_SV3CUT, IAN_BGS_SV3_MERGED_FILE, IAN_BGS_SV3_MERGED_NOY3_FILE]:
    table = Table.read(file, format='fits')

    # if the table doesn't have HALPHA or SFR, print a warning
    if 'HALPHA_EW' not in table.columns or 'SFR' not in table.columns or 'LOGMSTAR' not in table.columns:
        print(f"Warning: {file} does not have HALPHA_EW, SFR, or LOGMSTAR columns")
        halpha = None
        ssfr = None
    else:
        halpha = table['HALPHA_EW']
        ssfr = table['SFR'] / np.power(10, table['LOGMSTAR'])

    # if the table doesn't have DN4000_MODEL, print a warning
    if 'DN4000_MODEL' not in table.columns:
        print(f"Warning: {file} does not have DN4000_MODEL column")
        dn4000 = None
    else:
        dn4000 = table['DN4000_MODEL']

    quiescent = is_quiescent_BGS_dn4000(table['LOG_L_GAL'], dn4000, table['G_R_BEST'])
    #x, y, z, zz, quiescent, missing = is_quiescent_BGS_kmeans(table['LOG_L_GAL'], dn4000, halpha, ssfr, table['G_R_BEST'], model=QUIESCENT_MODEL_V2)
    table['QUIESCENT'] = quiescent

    table.write(file, format='fits', overwrite=True)
